|       | Description |
| ----------- | ----------- |
| Author      | Kamin Atsavasirilert (MSFE student at UIUC)|
| Date   | 10/30/2023        |
|Email|kamina2@illinois.edu|
|Idea| Use Selenium to get the news contents from Barron's |

In [1]:
from selenium import webdriver # allow launching browser
from selenium.webdriver.common.by import By # allow search with parameters
from selenium.webdriver.support.ui import WebDriverWait # allow waiting for page to load
from selenium.webdriver.support import expected_conditions as EC # determine whether the web page has loaded
from selenium.common.exceptions import TimeoutException # handling timeout situation
import time
import os
from random import uniform, random, choice, shuffle
import pandas as pd
from tqdm import tqdm
import pickle

In [2]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

In [ ]:
def create_webdriver():
    return webdriver.chrome.webdriver.WebDriver()

In [ ]:
email = '<YOUR_EMAIL>'
PW = '<PASSWORD>'

In [ ]:
browser = create_webdriver()
link = "https://sso.accounts.dowjones.com/login-page?client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&redirect_uri=https%3A%2F%2Fwww.barrons.com%2Fclient%2Fauth&response_type=code&scope=openid%20idp_id%20roles%20tags%20email%20given_name%20family_name%20uuid%20djid%20djUsername%20djStatus%20trackid%20prts%20updated_at%20created_at%20offline_access&ui_locales=en-us-x-barrons-223-2&nonce=5be85805-8aae-4e29-879d-6443fcc540a7&state=7jxTAzU1TvYYT8DZ.YXZXI-6hSh7SUwIahTuw8TWcpS5djZgQE4b5PIUveh8&resource=https%253A%252F%252Fwww.barrons.com%252F&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#!/signin"
browser.get(link)

In [ ]:
email_field = browser.find_element(By.ID, 'username')
email_field.send_keys(email)

In [ ]:
l = browser.find_element(By.XPATH, '//*[@id="basic-login"]/div[1]/form/div[2]/div[6]/div[1]/button[1]')
browser.execute_script("arguments[0].click();", l);

In [ ]:
pw_field = browser.find_element(By.XPATH, '//*[@id="password-login-password"]')
pw_field.send_keys(PW)

In [ ]:
login_button = browser.find_element(By.XPATH, '//*[@id="password-login"]/div/form/div/div[5]/div[1]/button')
browser.execute_script("arguments[0].click();", login_button);

In [ ]:
keyword = "economy"
link2 = f"https://www.barrons.com/search?query={keyword}&quotequery=80&isToggleOn=true&operator=OR&sort=relevance&duration=4y&startDate=2022%2F03%2F29&endDate=2023%2F10%2F16&source=barrons%2Cautowire&search_keywords=economy&search_statement_type=filtered"
# link2 = f"https://www.barrons.com/search?query={keyword}&quotequery=dwcf&isToggleOn=true&operator=OR&sort=relevance&duration=4y&startDate=2019%2F10%2F16&endDate=2023%2F10%2F16&source=barrons%2Cautowire&search_keywords=stock&search_statement_type=filtered"
browser.get(link2)

In [ ]:
links = []
for i in range(500):
    elems = browser.find_elements(By.XPATH,"//a[@href]")
    tmp_set = {elem.get_attribute("href") for elem in elems if "barrons.com/articles/" in elem.get_attribute("href")}
    links += list(tmp_set)
    browser.get(link2 + f"&page={i+2}")
    print("reaching page: ", i + 1, 'len(links):', len(links))

    time.sleep(choice([(random() + 0.5) * 20, (random() + 0.5) * 30]))

# News content extraction

In [ ]:
tmp_all_links = <LIST OF YOUR LINKS>

In [ ]:
cnt = 0
data = {'link': [], 'topic': [], 'time': [], 'header': [], 'content': []}
with tqdm(total = len(tmp_all_links)) as pbar:
    for ind, i in enumerate(tmp_all_links):
        pbar.update(1)
        browser.get(i)
        print('ind:',ind)
        print(i)
        tmp_id = i.split('/')[-1].split('?')[0]
#         print(tmp_id)
        try:
            time.sleep(1.5)
            tmp_date = browser.find_element(By.XPATH,f'//*[@id="{tmp_id}"]/div[1]/div[4]/div/p').text
            header = browser.find_elements(By.TAG_NAME, 'h1')[0].text
            contents = [i.text for i in browser.find_elements(By.TAG_NAME, 'P')]
            category = browser.find_elements(By.TAG_NAME, 'NAV')[0].text

        except:
            try: 
                time.sleep(1)
                tmp_date = browser.find_element(By.XPATH,f'//*[@id="{tmp_id}"]/div[1]/div[4]/div/p').text
                header = browser.find_elements(By.TAG_NAME, 'h1')[0].text
                contents = [i.text for i in browser.find_elements(By.TAG_NAME, 'P')]
                category = browser.find_elements(By.TAG_NAME, 'NAV')[0].text
            except:
                print('elem.text: broken')
                cnt += 1
                continue
        print('header:', header)
        data['link'].append(i)
        data['topic'].append(category)
        data['time'].append(tmp_date)
        data['header'].append(header)
        data['content'].append(contents)